# Demo for face verification app

# 1. Intro

We have completed training some model and store those models to `model_saved` folder. Now we load those models and use them to verify the face of a person.

The process will be as follow:

1. User register their face to the system through sacnning process. After we get sanning images, extract face then extract face embeddings and store them to database.

2. When user want to verify their face, we open the camera, capture the image, extract face embeddings and compare with the embeddings in database (with correspoding name use provide when login).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import random
import uuid
from PIL import Image
from mtcnn.mtcnn import MTCNN
from numpy import savez_compressed
import pickle

# For the Facenet model
import torch  # Ensure torch is imported here to avoid circular import issues
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms

2024-10-30 11:03:38.487991: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 11:03:38.508706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 11:03:38.532059: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 11:03:38.539135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 11:03:38.560981: I tensorflow/core/platform/cpu_feature_guar

Here, we defined the camera that the system will connect to:

- CAM_ID = 1 for laptop normal webcam
- CAM_ID = 3 for laptop IR webcam
- CAM_ID = 5 for external webcam


***Depend on each devices, these number can be different. Try out all number start from 0 and see which one is the correct one on your device.***


In [2]:
CAM_ID = 1

# 2. Enrollment process

In [3]:
# Create a application_data folder to store all app related data

os.makedirs('application_data', exist_ok=True)

# Inside this foilder, create a folder name validation_images to store all the images that are used for validation process
valiation_images = os.path.join('application_data', 'validation_images')
os.makedirs(valiation_images, exist_ok=True)

In [ ]:
# Connect to the camera and take pictures of the user for sacnning process
# Save the images in the validation_images folder, inside a subfolder with the user's name

# Function to capture images from webcam
def capture_images(user_name, store_location='application_data/validation_images'):
    user_folder = os.path.join(store_location, user_name)
    os.makedirs(user_folder, exist_ok=True)
    
    cap = cv2.VideoCapture(CAM_ID)
    print("Press 'p' to capture an image, 'q' to quit.")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow('Enrollment process, p to capture, q to quit', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('p'):
            img_name = f"{uuid.uuid4()}.jpg"
            img_path = os.path.join(user_folder, img_name)
            cv2.imwrite(img_path, frame)
            print(f"Image saved: {img_path}")
        elif key == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Ask for user name and capture images
user_name = input("Enter your name to register to system: ")
capture_images(user_name)

Press 'p' to capture an image, 'q' to quit.


In [5]:
# Now, we process the images to extract the face of a subfolder/person name in the validation_images folder
# then store the faces.npz right in that subfolder, using MTCNN to detect faces

# Function to detect faces and save to faces.npz
# Parameters:
# user_name: Name of the user whose images are to be processed
def detect_and_save_faces(user_name, store_location='application_data/validation_images'):
    user_folder = os.path.join(store_location, user_name)
    face_folder = os.path.join(user_folder, 'face') # A subfolder with person name already contains
    # sacnning images, so make a seperate `face` subfolder isnide that to sotre the faces.npz for better organization
    os.makedirs(face_folder, exist_ok=True)
    
    detector = MTCNN()
    faces = []
    
    for img_file in os.listdir(user_folder): # Loop through all the images in the user folder
        if img_file.endswith('.jpg'):
            img_path = os.path.join(user_folder, img_file)
            image = Image.open(img_path).convert('RGB')
            image_np = np.array(image)
            detections = detector.detect_faces(image_np)
            
            for i, detection in enumerate(detections):
                x, y, width, height = detection['box']
                face = image_np[y:y+height, x:x+width]
                face_image = Image.fromarray(face).resize((160, 160))
                face_array = np.array(face_image)
                faces.append(face_array)
    
    faces = np.array(faces)
    savez_compressed(os.path.join(face_folder, 'faces.npz'), faces)


# Call the function to detect faces and save to faces.npz
if user_name:  #username is input from the user at the previous step
    detect_and_save_faces(user_name)



The structure is as follow:

```plaintext
application_data
|
|───Validation_images
|   |───user1
|   |   |───face
|   |   |   └───faces.npz
|   |   |───image1.jpg
|   |   |───image2.jpg
|   |   |───...
```

In [6]:
# After the above steps, we have the faces.npz file for the user, from that file, we continue
# to extract the face embeddings

# Load the pre-trained FaceNet model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)


# Define a function to generate embeddings (this function is already defined in
# the Preprocessing Notebook, so we can just copy it here)
# Parameters:
# - image_array: a numpy array representing the image
def generate_embedding(image_array, model=facenet_model):
    # Convert numpy array to PIL image
    image = Image.fromarray(image_array)
    # Preprocess the image
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    # Generate embedding
    with torch.no_grad():
        embedding = model(img_tensor).cpu().numpy()
    return embedding

# Function to generate embeddings and save to embedding.npz
def generate_and_save_embeddings(user_name, store_location='application_data/validation_images'):
    user_folder = os.path.join(store_location, user_name)
    face_folder = os.path.join(user_folder, 'face')
    embedding_folder = os.path.join(user_folder, 'embeddings')
    if not os.path.exists(embedding_folder):
        os.makedirs(embedding_folder)
    
    data = np.load(os.path.join(face_folder, 'faces.npz'))
    faces = data['arr_0']
    embeddings = []
    
    for face in faces:
        embedding = generate_embedding(face)
        embeddings.append(embedding)
    
    embeddings = np.array(embeddings)
    savez_compressed(os.path.join(embedding_folder, 'embeddings.npz'), embeddings)

# Call the function to generate embeddings and save to embeddings.npz
if user_name: # if user_name not null
    generate_and_save_embeddings(user_name)


The structure is as follow:

```plaintext
application_data
|
|───Validation_images
|   |───user1
|   |   |───face
|   |   |   └───faces.npz
|   |   |
|   |   |───embeddings
|   |   |   └───embeddings.npz
|   |   |
|   |   |───image1.jpg
|   |   |───image2.jpg
|   |   |───...
```

The final goal is just the `embeddings.npz` file, other face.npz, images are just for vizuale the pipeline.

# 3. Verification process

In [10]:
# When use wnat to login, capture a image from the webcam when user press 'v' and 
# then compare that input image with the all the embeddings of the user to check if the user is the same person

# Function to capture a single image directly from webcam, return the frame object
def verify_user(username, detector, facenet_model):


    # Frist, check if user existed in the system
    if not os.path.exists(os.path.join('application_data/validation_images', username)):
        print("User not found")
        # Then return an empty list
        return []

    cap = cv2.VideoCapture(CAM_ID)
    # Declare a variable frame to store the captured image
    frame = None

    
    result = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow('Verify user. Press v to capture an image', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('v'):
            embedding_of_input = extract_face_and_generate_embedding(frame, detector, facenet_model)
            result = compare_embeddings(username,embedding_of_input, svm_model, scaler)
            break
    cap.release()
    cv2.destroyAllWindows()

    return result

   

# After capture image, extract the face from the image and generate the embeddings
def extract_face_and_generate_embedding(frame, detector, facenet_model):
    
    faces = []
    
    image = Image.fromarray(frame).convert('RGB')
    image = np.array(image)
    detections = detector.detect_faces(image)
    
    for i, detection in enumerate(detections):
        x, y, width, height = detection['box']
        face = image[y:y+height, x:x+width]
        face_image = Image.fromarray(face).resize((160, 160))
        face_array = np.array(face_image)
        faces.append(face_array)
    
    faces = np.array(faces)
    embedding = generate_embedding(faces[0], facenet_model) # Since we are capturing a single image, we only have one face
    return embedding

def generate_embedding(image_array, model=facenet_model):
    # Convert numpy array to PIL image
    image = Image.fromarray(image_array)
    # Preprocess the image
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    # Generate embedding
    with torch.no_grad():
        embedding = model(img_tensor).cpu().numpy()
    return embedding


# Function to compare the input image with all embeddings of the user using the model we pass in

def compare_embeddings(username, embedding_of_input, svm_model, scaler):
    user_folder = os.path.join('application_data/validation_images', username)
    embedding_folder = os.path.join(user_folder, 'embeddings')
    

    data = np.load(os.path.join(embedding_folder, 'embeddings.npz'))
    embeddings = data['arr_0']
    
    results = []
    
    for validation_embedding in embeddings:
        
        # Flatten the embeddings
        input_embedding_flat = embedding_of_input.flatten()
        validation_embedding_flat = validation_embedding.flatten()
        
        pair = np.concatenate((input_embedding_flat, validation_embedding_flat))
        pair_scaled = scaler.transform([pair])
        
        prediction = svm_model.predict(pair_scaled)
        probabilities = svm_model.predict_proba(pair_scaled)
        print("Compare with validation image, rresult is:", prediction[0], 'with confidence: ', probabilities)
        results.append(prediction[0])
    
    return results


# Main function

# Declare all model to use
detector = MTCNN()
svm_model = None
scaler = None
# Load the pre-trained FaceNet model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)



with open('./model_saved/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

with open('./model_saved/svm_model.pkl', 'rb') as f:
        svm_model = pickle.load(f)

# Prompt who are trying to login
username = input("Who are you ?")
result = []
if username: # prevent the case user enter nothing
    result = verify_user(username, detector, facenet_model)
    print(result)


# Now, based on the result, we can decide if the user is the same person or not with a threshold. 
# If the proportion of output 1 / total output is greater than a threshold, we can say the user is the same person
# Define a threshold
threshold = 0.8

# Calculate the proportion of positive identifications
positive_identifications = sum(result)
total_identifications = len(result)

if total_identifications == 0:
    print("No face detected in the input image or the user is not found in the system")
else: 
    proportion = positive_identifications / total_identifications

    # Determine if the user is the same person
    if proportion > threshold:
        print("User verified successfully.")
    else:
        print("User verification failed.")


Compare with validation image, rresult is: 1 with confidence:  [[0.45064455 0.54935545]]
Compare with validation image, rresult is: 1 with confidence:  [[0.47115633 0.52884367]]
Compare with validation image, rresult is: 1 with confidence:  [[0.53099406 0.46900594]]
Compare with validation image, rresult is: 1 with confidence:  [[0.5 0.5]]
Compare with validation image, rresult is: 1 with confidence:  [[0.5154288 0.4845712]]
Compare with validation image, rresult is: 1 with confidence:  [[0.52112091 0.47887909]]
Compare with validation image, rresult is: 1 with confidence:  [[0.5224137 0.4775863]]
Compare with validation image, rresult is: 1 with confidence:  [[0.46448314 0.53551686]]
Compare with validation image, rresult is: 1 with confidence:  [[0.48743148 0.51256852]]
Compare with validation image, rresult is: 1 with confidence:  [[0.51844597 0.48155403]]
Compare with validation image, rresult is: 1 with confidence:  [[0.5 0.5]]
Compare with validation image, rresult is: 1 with con

In [11]:
print(len(result))

33
